<a href="https://colab.research.google.com/github/msy7822-ux/diffusers-train-controlnet/blob/main/Diffusers_train_Controlnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title hugging face login

!pip install huggingface_hub
!huggingface-cli login

In [ ]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
# @title Install Library

!pip install git+https://github.com/huggingface/diffusers.git transformers accelerate xformers==0.0.16 wandb
!wandb login

In [ ]:
# @title clone diffusers repo

!ls
!git clone https://github.com/huggingface/diffusers.git /content/drive/MyDrive/diffusers
%cd /content/drive/MyDrive/diffusers

In [ ]:
# @title change workspace directory

%cd /content/drive/MyDrive/diffusers

In [ ]:
# @title train controlnet

%cd /content/drive/MyDrive/diffusers/examples/controlnet

!pip install -r requirements.txt
!pip install datasets

In [ ]:
# @title HF Script

from datasets import Dataset, load_dataset
from huggingface_hub import HfApi, HfFolder
import os
from PIL import Image

conditioning_paths = [
    "/content/drive/MyDrive/diffusers/examples/datasets/conditioning/depth1.png"
]

def create_dataset_from_images(path):
    """
    画像フォルダから画像のパスを読み込んでDatasetを作成する
    """

    image_folder = os.path.join(path, "image")
    conditioning_folder = os.path.join(path, "conditioning")

    image = [Image.open(os.path.join(image_folder, f)) for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))]
    conditioning = [Image.open(os.path.join(conditioning_folder, f)) for f in os.listdir(conditioning_folder) if os.path.isfile(os.path.join(conditioning_folder, f))]
    caption = ["simple, single point perspective, one point perspective, anime,"]*len(conditioning_paths)

    dataset = Dataset.from_dict({'image': image, 'conditioning': conditioning, 'caption': caption})
    return dataset

def upload_dataset_to_hub(dataset, dataset_name, organization=None):
    """
    DatasetをHugging Face Hubにアップロードする
    """
    # Hugging Faceの認証トークンを取得
    api = HfApi()
    token = HfFolder.get_token()
    if token is None:
        raise ValueError("Hugging Faceの認証トークンが見つかりません。huggingface-cliでログインしてください。")

    # データセットをアップロード
    if organization:
        repo_id = f"{organization}/{dataset_name}"
    else:
        repo_id = dataset_name
        print(repo_id, token)
    dataset.push_to_hub(repo_id, token=token)

# 画像が格納されているローカルフォルダのパス
image_folder = '/content/drive/MyDrive/diffusers/examples/datasets'

# Datasetオブジェクトを作成
image_dataset = create_dataset_from_images(image_folder)

# DatasetをHugging Face Hubにアップロード
upload_dataset_to_hub(image_dataset, 'newDatasets', 'msy78')


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]